In [1]:
import altair as alt
import pandas as pd


# Enable VegaFusion transformer to handle large datasets
alt.data_transformers.enable('vegafusion')

crime_data = pd.read_csv('pd_data_clean.csv')
code_data = pd.read_excel('rmsoffensecodes.xlsx')

# Define refined categories based on offense descriptions
violent_keywords = ['ASSAULT', 'ROBBERY', 'HOMICIDE', 'BATTERY', 'DOMESTIC', 'SEXUAL']
car_keywords = ['MOTOR VEHICLE', 'MV', 'CAR', 'TRAFFIC', 'ACCIDENT', 'HIT AND RUN']
theft_keywords = ['LARCENY', 'BURGLARY', 'THEFT', 'SHOPLIFTING', 'STOLEN']

In [2]:
# Function to categorize offenses
def categorize_offense(name):
    name_upper = name.upper()
    if any(keyword in name_upper for keyword in violent_keywords):
        return 'Violent Crime'
    elif any(keyword in name_upper for keyword in car_keywords):
        return 'Car Incident'
    elif any(keyword in name_upper for keyword in theft_keywords):
        return 'Theft'
    return 'Other'

In [3]:
# Apply categorization to the offense codes
code_data['Category'] = code_data['NAME'].apply(categorize_offense)

# Merge the refined categories back into the main crime dataset
crime_data = crime_data.merge(
    code_data[['CODE', 'Category']],
    left_on='OFFENSE_CODE',
    right_on='CODE',
    how='left'
)

# Filter the dataset to include only relevant categories
filtered_data = crime_data[crime_data['Category'].isin(['Violent Crime', 'Car Incident', 'Theft'])]

# Display the counts of each category for verification
print(filtered_data['Category'].value_counts())


# Convert OCCURRED_ON_DATE to datetime for proper handling
filtered_data['OCCURRED_ON_DATE'] = pd.to_datetime(filtered_data['OCCURRED_ON_DATE'])

# 1. Total of Selected Crimes per Month
monthly_crimes = (
    filtered_data
    .groupby('MONTH')
    .size()
    .reset_index(name='Count')
)

chart_monthly = alt.Chart(monthly_crimes).mark_bar().encode(
    x=alt.X('MONTH:O', title='Month', sort='ascending'),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    tooltip=['MONTH', 'Count']
).properties(
    title='Total Monthly Amounts of Violent Crimes, Car Incidents, and Thefts ',
    width=600,
    height=400
)
chart_monthly

# Save the Altair visualization to an HTML file
output_html = "monthly.html"
chart_monthly.save(output_html)


Category
Theft            41627
Car Incident     31985
Violent Crime    20904
Name: count, dtype: int64


/var/folders/sb/5p2wcvwd2fld66n82t9v8vwh0000gn/T/ipykernel_6723/3938445466.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['OCCURRED_ON_DATE'] = pd.to_datetime(filtered_data['OCCURRED_ON_DATE'])


In [4]:
# Times of day for each crime plot 
hourly_count = (
    filtered_data
    .groupby('HOUR')
    .size()
    .reset_index(name='Count')
)

chart_hourly = alt.Chart(hourly_count).mark_line(point=True, color='coral').encode(
    x=alt.X('HOUR:Q', title='Hour (24-hour clock)'),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    tooltip=['HOUR', 'Count']
).properties(
    title='Selected Crimes by Hour of Day',
    width=600,
    height=400
)
chart_hourly

# Save the Altair visualization to an HTML file
output_html = "total_hourly.html"
chart_hourly.save(output_html)

In [5]:
# Hourly absolute counts
hourly_count = (
    filtered_data
    .groupby(['HOUR', 'Category'])
    .size()
    .reset_index(name='Count')
)

chart_hourly_absolute = alt.Chart(hourly_count).mark_line(point=True).encode(
    x=alt.X('HOUR:Q', title='Hour (24-hour clock)'),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    color='Category:N',
    tooltip=['HOUR', 'Category', 'Count']
).properties(
    title='Crimes by Hour of Day (Absolute Counts)',
    width=600,
    height=400
)

chart_hourly_absolute

# Save the Altair visualization to an HTML file
output_html = "hourly.html"
chart_hourly_absolute.save(output_html)

In [6]:
# Hourly percentages
hourly_percentage = (
    hourly_count
    .assign(Percentage=lambda df: df.groupby('HOUR')['Count'].transform(lambda x: x / x.sum() * 100))
)

chart_hourly_percentage = alt.Chart(hourly_percentage).mark_line(point=True).encode(
    x=alt.X('HOUR:Q', title='Hour (24-hour clock)'),
    y=alt.Y('Percentage:Q', title='Percentage of Incidents'),
    color='Category:N',
    tooltip=['HOUR', 'Category', 'Percentage']
).properties(
    title='Crimes by Hour of Day (Normalized Percentages)',
    width=600,
    height=400
)

chart_hourly_percentage

# Save the Altair visualization to an HTML file
output_html = "hourly_normal.html"
chart_hourly_percentage.save(output_html)

In [7]:
monthly_count = (
    filtered_data.groupby(['MONTH', 'Category'])
    .size()
    .reset_index(name='Count')
)

# Monthly absolute counts as line graph
chart_monthly = alt.Chart(monthly_count).mark_line(point=True).encode(
    x=alt.X('MONTH:O', title='Month', sort='ascending'),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    color='Category:N',
    tooltip=['MONTH', 'Category', 'Count']
).properties(
    title='Monthly Average of Selected Crimes (Absolute Counts)',
    width=600,
    height=400
)

chart_monthly

# Save the Altair visualization to an HTML file
output_html = "monthly_normalized.html"
chart_monthly.save(output_html)

In [8]:
# Add a column for the total counts per month
monthly_count['Total_Count'] = monthly_count.groupby('MONTH')['Count'].transform('sum')

# Calculate the percentage
monthly_count['Percentage'] = (monthly_count['Count'] / monthly_count['Total_Count']) * 100

# Rename the DataFrame for clarity
monthly_percentage = monthly_count

# Monthly percentages as line graph
chart_monthly_percentage = alt.Chart(monthly_percentage).mark_line(point=True).encode(
    x=alt.X('MONTH:O', title='Month', sort='ascending'),
    y=alt.Y('Percentage:Q', title='Percentage of Incidents'),
    color='Category:N',
    tooltip=['MONTH', 'Category', 'Percentage']
).properties(
    title='Monthly Average of Selected Crimes (Normalized Percentages)',
    width=600,
    height=400
)

chart_monthly_percentage

# Save the Altair visualization to an HTML file
output_html = "monthly_normalized.html"
chart_monthly_percentage.save(output_html)

In [9]:
# Calculate day-of-week counts for each crime category
day_of_week_count = (
    filtered_data.groupby(['DAY_OF_WEEK', 'Category'])
    .size()
    .reset_index(name='Count')
)

# Day of the week absolute counts as line graph
chart_day_of_week = alt.Chart(day_of_week_count).mark_line(point=True).encode(
    x=alt.X('DAY_OF_WEEK:O', title='Day of the Week', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    color='Category:N',
    tooltip=['DAY_OF_WEEK', 'Category', 'Count']
).properties(
    title='Crime Count by Day of the Week (Absolute Counts)',
    width=600,
    height=400
)

chart_day_of_week

# Save the Altair visualization to an HTML file
output_html = "day_of_week.html"
chart_day_of_week.save(output_html)

In [10]:
# Add a column for the total counts per day of the week
day_of_week_count['Total_Count'] = day_of_week_count.groupby('DAY_OF_WEEK')['Count'].transform('sum')

# Calculate the percentage
day_of_week_count['Percentage'] = (day_of_week_count['Count'] / day_of_week_count['Total_Count']) * 100

# Rename the DataFrame for clarity
day_of_week_percentage = day_of_week_count

# Day of the week percentages as line graph
chart_day_of_week_percentage = alt.Chart(day_of_week_percentage).mark_line(point=True).encode(
    x=alt.X('DAY_OF_WEEK:O', title='Day of the Week', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']),
    y=alt.Y('Percentage:Q', title='Percentage of Incidents'),
    color='Category:N',
    tooltip=['DAY_OF_WEEK', 'Category', 'Percentage']
).properties(
    title='Crime Count by Day of the Week (Normalized Percentages)',
    width=600,
    height=400
)

chart_day_of_week_percentage

# Save the Altair visualization to an HTML file
output_html = "day_of_week_normalized.html"
chart_day_of_week_percentage.save(output_html)

In [11]:
import altair as alt
import pandas as pd

# Step 1: Aggregate the data
hourly_data = filtered_data.groupby(['Category', 'HOUR']).size().reset_index(name='Count')
daily_data = filtered_data.groupby(['Category', 'DAY_OF_WEEK', 'HOUR']).size().reset_index(name='Count')
monthly_data = filtered_data.groupby(['Category', 'MONTH']).size().reset_index(name='Count')

# Step 2: Create selectors (dropdown for crime category)
category_selector = alt.selection_single(
    fields=['Category'],
    bind=alt.binding_select(options=filtered_data['Category'].unique().tolist()),
    name="Select Crime Type"
)

# Step 3: Create individual visualizations

# Line chart: Crimes by hour
hourly_chart = alt.Chart(hourly_data).mark_line(point=True).encode(
    x=alt.X('HOUR:Q', title='Hour (24-hour clock)'),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    color='Category:N',
    tooltip=['HOUR', 'Count'],
).transform_filter(
    category_selector
).properties(
    title="Crimes by Hour of Day",
    width=600,
    height=300
)

# Heatmap: Crimes by day and hour
heatmap = alt.Chart(daily_data).mark_rect().encode(
    x=alt.X('HOUR:O', title='Hour (24-hour clock)'),
    y=alt.Y('DAY_OF_WEEK:O', title='Day of the Week', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']),
    color=alt.Color('Count:Q', scale=alt.Scale(scheme='redblue', reverse=True), title='Number of Incidents'),
    tooltip=['DAY_OF_WEEK', 'HOUR', 'Count'],
).transform_filter(
    category_selector
).properties(
    title="Crimes by Day of Week and Hour",
    width=600,
    height=300
)

# Bar chart: Crimes by month
monthly_chart = alt.Chart(monthly_data).mark_bar().encode(
    x=alt.X('MONTH:O', title='Month'),
    y=alt.Y('Count:Q', title='Number of Incidents'),
    color='Category:N',
    tooltip=['MONTH', 'Count'],
).transform_filter(
    category_selector
).properties(
    title="Crimes by Month",
    width=600,
    height=300
)

# Step 4: Combine the charts
dashboard = (hourly_chart & heatmap & monthly_chart).add_selection(
    category_selector
)

dashboard

# Save the Altair visualization to an HTML file
output_html = "crime_dashboard.html"
dashboard.save(output_html)

/var/folders/sb/5p2wcvwd2fld66n82t9v8vwh0000gn/T/ipykernel_6723/1999549734.py:10: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  category_selector = alt.selection_single(
/var/folders/sb/5p2wcvwd2fld66n82t9v8vwh0000gn/T/ipykernel_6723/1999549734.py:61: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  dashboard = (hourly_chart & heatmap & monthly_chart).add_selection(


In [12]:
# Step 1: Aggregate data for the heatmap
global_heatmap_data = filtered_data.groupby(['DAY_OF_WEEK', 'HOUR']).size().reset_index(name='Total Count')

# Step 2: Aggregate data for the bar chart
breakdown_data = filtered_data.groupby(['DAY_OF_WEEK', 'HOUR', 'Category']).size().reset_index(name='Count')

# Step 3: Create the selection
time_selector = alt.selection_point(
    fields=['DAY_OF_WEEK', 'HOUR'],
    on='click',
    empty='none',
    name='Select Time'
)

# Step 4: Create the global heatmap
global_heatmap = alt.Chart(global_heatmap_data).mark_rect().encode(
    x=alt.X('HOUR:O', title='Hour (24-hour clock)'),
    y=alt.Y('DAY_OF_WEEK:O', title='Day of the Week', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']),
    color=alt.Color(
        'Total Count:Q',
        scale=alt.Scale(scheme='redblue', reverse=True),  # Blue-to-Red color scheme
        title='Total Incidents'
    ),
    tooltip=['DAY_OF_WEEK', 'HOUR', 'Total Count']
).add_params(
    time_selector
).properties(
    title="Crime Incidents Heatmap by Day and Hour (All Categories)",
    width=600,
    height=400
)

# Step 5: Create the dynamic title for the bar chart using transform_calculate
dynamic_title = alt.Chart(breakdown_data).transform_calculate(
    title="'Crime Breakdown for ' + datum.DAY_OF_WEEK + 'Between the Hours of ' + round(datum.HOUR) + ' and ' + round(datum.HOUR + 1)"
).mark_text(
    align='center',
    fontSize=14,
    fontWeight='bold'
).encode(
    text=alt.condition(
        time_selector,
        'title:N',  # Use the calculated title field when selection is active
        alt.value("")  # Hide the title when no selection
    )
).properties(
    width=600,
    height=30
)

# Step 6: Create the bar chart for crime type breakdown
crime_breakdown_chart = alt.Chart(breakdown_data).mark_bar().encode(
    x=alt.X('Category:N', title='Crime Type', axis=alt.Axis(labelAngle=0)),  # Horizontal labels
    y=alt.Y('Count:Q', title='Number of Incidents'),
    color=alt.Color('Category:N', legend=None),
    tooltip=['Category', 'Count']
).transform_filter(
    time_selector
).properties(
    width=600,
    height=300
)

# Step 7: Combine the title and bar chart
bar_chart_with_title = alt.vconcat(dynamic_title, crime_breakdown_chart)

# Step 8: Combine the heatmap and bar chart
interactive_dashboard = global_heatmap & bar_chart_with_title

interactive_dashboard

# Save the Altair visualization to an HTML file
output_html = "day_hour_dashboard.html"
interactive_dashboard.save(output_html)